In [2]:
# E:\MyJPY\Datas\风控模型
# https://www.kesci.com/mw/project/5ff1717d840381003b00585d/content

信贷业务，就是贷款业务，是商业银行和互联网金融公司最重要的资产业务和主要赢利手段。

通过放宽收回本金和利息，扣除成本后获得利润。
贷款平台预测有信贷需求用户的还款情况，然后将本金借贷给还款概率大的用户
风控就是对用户的信用风险进行管理和规避，对于预测信用较差的人，不向其放款，即便放款，也会是较小的贷款额度和较高的利率

信贷领域有两类风险

信用风险：借款人的还款能力和还款意愿在贷款后出现问题的风险
欺诈风险：借款人压根没想还钱，以咋骗为目的
风控业务主要针对这两类风险

信用评分系统：针对信用风险
反欺诈系统：针对欺诈风险
基于机器学习的人工智能风控模型对比传统人工审批

人工审批：效率低，对人员业务能力要求高，不适合金融零售业务场景
机器学习模型：批量、迅速、准确，同时处理大量贷款请求（几十万、上百万、天）
现金贷：申请借款->放款给客户->客户还款
消费贷：信用卡、白条等产品，有账单日，还款日
申请消费贷->额度授信->客户使用消费贷消费

风控相关术语介绍
DPD:Day past due 逾期天数 DPD0为到期当日，DPD1为逾期一日，DPD7为逾期一周
FPD:first time past due 首次逾期天数
FSTQPD:首次，二次，三次，四次 逾期天数
M1:逾期【1,30】天
M1+：逾期【30，inf】天
default:坏账
delinquency:拖欠
flow rate:流动率，一般指M1向M2，M2向M3专一的比例
bad rate:坏账率，当月不良资产数/总资产数
vintage:账龄分析

风控业务案例
数据集说明：

从开源数据改造而来，基本反映真实业务数据
销售，客服可以忽略
账单周期，放款日期
账单金额-实际金额=未收金额
应付日期为还款时间
账期分为两种 60天和90天
实际到账日为空白，说明没还钱

In [3]:
#代码实现
import pandas as pd
import datetime
from pyecharts.charts import *
from pyecharts import options as opts
df1=pd.read_excel(r'E:\MyJPY\Datas\风控模型\业务数据.xls')
#要使用原始数据构建新指标，所以保留原始数据，copy新的数据，在新的数据中创建新指标
df2=df1.copy()
df2.head()

,销售,账单状态,账单周期,账单金额,开票金额,实收金额,未收金额,预计付款日,应付日期,商务催收日期,账期,实际到账日,开票日期,客服
0,s101,未确认,2019-05,29805.0,NaN,NaN,NaN,2019-07-31,2019-07-31,2019-08-15,60,NaN,NaN,a201
1,s102,未确认,2019-05,1572.6,NaN,NaN,NaN,2019-07-31,2019-07-31,2019-08-15,60,NaN,NaN,a202
2,s103,已确认,2019-04,487551.2,487551.2,NaN,487551.2,2019-06-30,2019-06-30,2019-07-15,60,NaN,05-16,a203
3,s104,已确认,2019-04,378835.0,378835.0,NaN,378835.0,2019-07-31,2019-07-31,2019-08-15,90,NaN,05-08,a204
4,s105,已确认,2019-04,326866.0,326866.0,NaN,326866.0,2019-07-31,2019-07-31,2019-08-15,90,NaN,05-10,a205


In [4]:
#查看数据基本情况
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5257 entries, 0 to 5256
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   销售      5257 non-null   object 
 1   账单状态    5257 non-null   object 
 2   账单周期    5257 non-null   object 
 3   账单金额    5257 non-null   float64
 4   开票金额    5010 non-null   float64
 5   实收金额    4470 non-null   float64
 6   未收金额    5010 non-null   float64
 7   预计付款日   5256 non-null   object 
 8   应付日期    5257 non-null   object 
 9   商务催收日期  5257 non-null   object 
 10  账期      5257 non-null   int64  
 11  实际到账日   4387 non-null   object 
 12  开票日期    4996 non-null   object 
 13  客服      5257 non-null   object 
dtypes: float64(4), int64(1), object(9)
memory usage: 575.1+ KB


In [5]:
df2.describe()

,账单金额,开票金额,实收金额,未收金额,账期
count,5.257000e+03,5.010000e+03,4.470000e+03,5.010000e+03,5257.000000
mean,4.073241e+04,4.096896e+04,4.082419e+04,4.684636e+03,64.539661
std,8.176172e+04,8.007245e+04,7.970628e+04,2.888464e+04,15.622765
min,0.000000e+00,2.500000e+01,0.000000e+00,0.000000e+00,0.000000
25%,5.103000e+03,5.300000e+03,5.112250e+03,0.000000e+00,60.000000
50%,1.436500e+04,1.486560e+04,1.434000e+04,0.000000e+00,60.000000
75%,4.178000e+04,4.220250e+04,4.170750e+04,0.000000e+00,75.000000
max,1.508796e+06,1.356215e+06,1.301665e+06,1.277098e+06,90.000000


In [6]:
#数据处理，填充缺失值，将日期时间类型转换成datetime类型
#获取最大的日期，作为当前时间
today_time=pd.to_datetime(df2.实际到账日.fillna('0').max())

df2['实收金额']=df2.实收金额.fillna(0)
df2['开票金额']=df2.开票金额.fillna(0)
df2['未收金额']=df2.未收金额.fillna(0)

df2['账单周期']=pd.to_datetime(df2.账单周期)
df2['应付日期']=pd.to_datetime(df2.应付日期)

df2['实际到账日']=pd.to_datetime(df2.实际到账日).fillna(today_time)

In [7]:
#为了后续计算，在原始数据基础上构造新的字段：是否逾期，是否逾期90天，未收金额2（校验原始数据中的未收金额），当前逾期天数，历史逾期天数
df2['是否到期']=df2.apply(lambda x: 0 if x.应付日期 > today_time else 1,axis=1)
df2['是否到期90天']=(today_time - df2.应付日期).map(lambda x:1 if x.days >= 90 else 0)
df2['未收金额2']=(df2.账单金额-df2.实收金额)

df2['历史逾期天数']=df2.apply(lambda x:(x.实际到账日-x.应付日期).days if x.未收金额2 == 0 else (today_time - x.应付日期).days,axis=1)
16257820236

16257820236

In [8]:
df3=df2.copy()
#创建'账单季度'字段，将日期转换成季度
df3['账单季度']=df3['账单周期'].map(lambda x: x.to_period('Q'))
#提取2017年3季度到2018年4季度数据
df3=df3[(df3['账单季度']<='2018Q4') & (df3['账单季度']>='2017Q3')]
df3.shape

(3856, 19)

In [9]:
#按照季度统计账单金额，到期金额，和逾期金额
#账单金额
fn1=df3.groupby('账单季度')[['账单金额']].sum()
fn1.columns=['账单金额']
fn1

,账单金额
账单季度,
2017Q3,8247952.62
2017Q4,11643604.99
2018Q1,17149674.79
2018Q2,31097661.29
2018Q3,38292071.12
2018Q4,51963089.64


In [10]:
#90天到期金额
df4=df3[(df3.是否到期90天 == 1)]
fn2=df4.groupby('账单季度')[['账单金额']].sum()
fn2.columns=['到期金额']
fn2

,到期金额
账单季度,
2017Q3,8247952.62
2017Q4,11643604.99
2018Q1,17149674.79
2018Q2,31097661.29
2018Q3,38292071.12
2018Q4,28265677.59


In [11]:
df4=df3[(df3.是否到期90天 == 1)]
fn3=df4.groupby('账单季度')[['未收金额2']].sum()
fn3.columns=['当前逾期90+金额']
fn3

,当前逾期90+金额
账单季度,
2017Q3,63883.0
2017Q4,57380.0
2018Q1,64283.0
2018Q2,106930.0
2018Q3,412920.1
2018Q4,304183.0


In [12]:
#合并数据计算逾期率
dfs=[fn1,fn2,fn3]
final1=pd.concat(dfs,axis=1)
final1

,账单金额,到期金额,当前逾期90+金额
账单季度,,,
2017Q3,8247952.62,8247952.62,63883.0
2017Q4,11643604.99,11643604.99,57380.0
2018Q1,17149674.79,17149674.79,64283.0
2018Q2,31097661.29,31097661.29,106930.0
2018Q3,38292071.12,38292071.12,412920.1
2018Q4,51963089.64,28265677.59,304183.0


In [13]:
final1['90+净坏账率']=round(final1['当前逾期90+金额']/final1.到期金额,3)
final1

,账单金额,到期金额,当前逾期90+金额,90+净坏账率
账单季度,,,,
2017Q3,8247952.62,8247952.62,63883.0,0.008
2017Q4,11643604.99,11643604.99,57380.0,0.005
2018Q1,17149674.79,17149674.79,64283.0,0.004
2018Q2,31097661.29,31097661.29,106930.0,0.003
2018Q3,38292071.12,38292071.12,412920.1,0.011
2018Q4,51963089.64,28265677.59,304183.0,0.011


In [14]:
#pyecharts绘图
bar=(
    Bar()
    .add_xaxis(list(final1.index.values.astype(str)))
    .add_yaxis(
        "账单金额",
        list(final1.账单金额),
        yaxis_index=0,
        color='#5793f3',
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="90+净坏账率"),
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name='90+净坏账率',
            type_='value',
            min_=0,
            max_=0.014,
            position='right',
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="d14a61")
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
)
line=(
    Line()
    .add_xaxis(list(final1.index.values.astype(str)))
    .add_yaxis(
        "90+净坏账率",
        list(final1['90+净坏账率']),
        yaxis_index=1,
        color='#675bba',
        label_opts=opts.LabelOpts(is_show=False),
    )
)
bar.overlap(line).render_notebook()

In [15]:
#计算每个季度的60天账单入催金额，90天账单入催金额
#60天账期的账单金额
df4=df3[(df3.账期 == 60) &(df3.是否到期 == 1)]
fn1=df4.groupby('账单季度')[['账单金额']].sum()
fn1.columns=['60天账期的账单金额']
#60天账期的入催金额
df4=df3[(df3.账期==60) & (df3.是否到期 == 1) & (df3.历史逾期天数>0)]
fn2=df4.groupby('账单季度')[['未收金额2']].sum()
fn2.columns=['60天账期的入催金额']
#90天账期的账单金额
df4=df3[(df3.账期==90) & (df3.是否到期==1)]
fn3=df4.groupby('账单季度')[['账单金额']].sum()
fn3.columns=['90天账期的账单金额']
#90天账期的入催金额
df4=df3[(df3.账期==90)&(df3.是否到期==1)&(df3.历史逾期天数>0)]
fn4=df4.groupby('账单季度')[['未收金额2']].sum()
fn4.columns=['90天账期的入催金额']

In [16]:
#计算入催率
dfs=[fn1,fn2,fn3,fn4]
final2=pd.concat(dfs,axis=1)
final2['60天账期入催率']=round(final2['60天账期的入催金额']/final2['60天账期的账单金额'],3)
final2['90天账期入催率']=round(final2['90天账期的入催金额']/final2['90天账期的账单金额'],3)
final2

,60天账期的账单金额,60天账期的入催金额,90天账期的账单金额,90天账期的入催金额,60天账期入催率,90天账期入催率
账单季度,,,,,,
2017Q3,4854770.94,36983.0,2769264.0,1900.0,0.008,0.001
2017Q4,6737327.99,52750.0,3921491.0,0.0,0.008,0.000
2018Q1,12106356.79,62460.0,4244304.0,800.0,0.005,0.000
2018Q2,19234086.87,13590.0,8427775.0,0.0,0.001,0.000
2018Q3,22830710.42,380265.1,9835629.0,8235.0,0.017,0.001
2018Q4,26337959.52,584789.5,17706430.0,325141.0,0.022,0.018


In [17]:
#pyecharts绘图
line=(
    Line()
    .add_xaxis(list(final1.index.values.astype(str)))
    .add_yaxis(
        '60天账期入催率',
        list(final2['60天账期入催率']),
        yaxis_index=0,
        color='#675bba',
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title='不同账期入催率'),
    )
    .add_xaxis(list(final1.index.values.astype(str)))
    .add_yaxis(
        '90天账期入催率',
        list(final2['90天账期入催率']),
        yaxis_index=0,
        color='#d14a61',
        label_opts=opts.LabelOpts(is_show=False)
    )
)
line.render_notebook()

In [18]:
#不同逾期天数的回收情况
df6=df3[(df3.未收金额2==0)&(df3.是否到期==1)].copy()
#使用cut,将数据按照逾期天数分箱，然后添加分箱之后结果标签
df6['历史逾期天数']=pd.cut(df6['历史逾期天数'],bins=[-999,0,5,10,15,20,30,60,90,999],right=True,
    labels=['0','1-5','6-10','11-15','16-20','21-30','31-60','61-90','91+'])
final3=df6.groupby('历史逾期天数')[['账期']].count()
final3.columns=['回收账单数']
final3

,回收账单数
历史逾期天数,
0,2400
1-5,358
6-10,235
11-15,215
16-20,92
21-30,189
31-60,156
61-90,60
91+,88


In [19]:
#pyecharts绘图
ydata=final3['回收账单数'].values.tolist()
bar=(
    Bar()
    .add_xaxis(list(final3.index.values.tolist()))
    .add_yaxis('收回账单数',ydata,yaxis_index=0, color='#675bba')
    .set_global_opts(
        title_opts=opts.TitleOpts(title='不同逾期天数的已收回账单数'),
    )
)
bar.render_notebook()

### 风控建模概述

In [21]:
#加载数据
import pandas as pd
import numpy as np
data=pd.read_excel(r'E:\MyJPY\Datas\风控模型\rule_data.xlsx')
data.head(5)

,uid,oil_actv_dt,create_dt,total_oil_cnt,pay_amount_total,class_new,bad_ind,oil_amount,discount_amount,sale_amount,amount,pay_amount,coupon_amount,payment_coupon_amount,channel_code,oil_code,scene,source_app,call_source
0,A8217710,2018-08-19,2018-08-17,275.0,48295495.4,B,0,3308.56,1760081.0,1796001.0,1731081.0,8655401.0,1.0,1.0,1,3,2,0,3
1,A8217710,2018-08-19,2018-08-16,275.0,48295495.4,B,0,4674.68,2487045.0,2537801.0,2437845.0,12189221.0,1.0,1.0,1,3,2,0,3
2,A8217710,2018-08-19,2018-08-15,275.0,48295495.4,B,0,1873.06,977845.0,997801.0,961845.0,4809221.0,1.0,1.0,1,2,2,0,3
3,A8217710,2018-08-19,2018-08-14,275.0,48295495.4,B,0,4837.78,2526441.0,2578001.0,2484441.0,12422201.0,1.0,1.0,1,2,2,0,3
4,A8217710,2018-08-19,2018-08-13,275.0,48295495.4,B,0,2586.38,1350441.0,1378001.0,1328441.0,6642201.0,1.0,1.0,1,2,2,0,3


In [22]:
#查看class_new
data.class_new.unique()

array(['B', 'E', 'C', 'A', 'D', 'F'], dtype=object)

In [24]:
#原始数据的特征太少，考虑在原始特征基础商衍生出一些新的特征来，将特征分成三类分别处理
#数值类型变量，按照id分组后，采用多种方式聚合，衍生新特征
#分类类型变量，按照id分组后，聚合查询条目数量，衍生新特征
#其他：日期时间类型，是否违约（标签）,用户评级等不做特征衍生处理

org_list=['uid','create_dt','oil_actv_dt','class_new','bad_ind']
agg_list=['oil_amount','discount_amount','sale_amount','amount','pay_amount','coupon_amount','payment_coupon_amount']
count_list=['channel_code','oil_code','scene','source_app','call_source']

In [25]:
#创建数据副本，保留底表，并查看缺失情况
df=data[org_list].copy()
df[agg_list]=data[agg_list].copy()
df[count_list]=data[count_list].copy()
df.isna().sum()

uid                         0
create_dt                4944
oil_actv_dt                 0
class_new                   0
bad_ind                     0
oil_amount               4944
discount_amount          4944
sale_amount              4944
amount                   4944
pay_amount               4944
coupon_amount            4944
payment_coupon_amount    4946
channel_code                0
oil_code                    0
scene                       0
source_app                  0
call_source                 0
dtype: int64

In [26]:
#查看数值型变量的分布情况
df.describe()

,bad_ind,oil_amount,discount_amount,sale_amount,amount,pay_amount,coupon_amount,payment_coupon_amount,channel_code,oil_code,scene,source_app,call_source
count,50609.000000,45665.000000,4.566500e+04,4.566500e+04,4.566500e+04,4.566500e+04,45665.000000,45663.000000,50609.000000,50609.000000,50609.000000,50609.000000,50609.000000
mean,0.017764,425.376107,1.832017e+05,1.881283e+05,1.808673e+05,9.043344e+05,0.576853,149.395397,1.476378,1.617894,1.906519,0.306072,2.900729
std,0.132093,400.596244,2.007574e+05,2.048742e+05,1.977035e+05,9.885168e+05,0.494064,605.138823,1.511470,3.074166,0.367280,0.893682,0.726231
min,0.000000,1.000000,0.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,175.440000,6.039100e+04,6.200100e+04,5.976100e+04,2.988010e+05,0.000000,1.000000,1.000000,0.000000,2.000000,0.000000,3.000000
50%,0.000000,336.160000,1.229310e+05,1.279240e+05,1.209610e+05,6.048010e+05,1.000000,1.000000,1.000000,0.000000,2.000000,0.000000,3.000000
75%,0.000000,557.600000,2.399050e+05,2.454010e+05,2.360790e+05,1.180391e+06,1.000000,100.000000,1.000000,0.000000,2.000000,0.000000,3.000000
max,1.000000,7952.820000,3.916081e+06,3.996001e+06,3.851081e+06,1.925540e+07,1.000000,50000.000000,6.000000,9.000000,2.000000,3.000000,4.000000


In [27]:
#缺失值填充
#对create_dt做补全，用oil_actv_dt来填补
#截取申请时间和放款时间不超过6个月的数据（考虑数据时效性）
def time_isna(x,y):
    if str(x) == 'NaT':
        x = y
    return x
df2=df.sort_values(['uid','create_dt'],ascending=False)
df2['create_dt']=df2.apply(lambda x:time_isna(x.create_dt, x.oil_actv_dt),axis=1)
df2['dtn']=(df2.oil_actv_dt - df2.create_dt).apply(lambda x: x.days)
df=df2[df2['dtn']<180]
df.head()

,uid,create_dt,oil_actv_dt,class_new,bad_ind,oil_amount,discount_amount,sale_amount,amount,pay_amount,coupon_amount,payment_coupon_amount,channel_code,oil_code,scene,source_app,call_source,dtn
50608,B96436391985035703,2018-10-08,2018-10-08,B,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,9,2,3,4,0
50607,B96436391984693397,2018-10-11,2018-10-11,E,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,9,2,3,4,0
50606,B96436391977217468,2018-10-17,2018-10-17,B,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,9,2,3,4,0
50605,B96436391976480892,2018-09-28,2018-09-28,B,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,9,2,3,4,0
50604,B96436391972106043,2018-10-19,2018-10-19,A,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,9,2,3,4,0


In [28]:
#将用户按照id编号排序，并保留最近一次申请时间，确保每个用户有一条记录
base=df[org_list]
base['dtn']=df['dtn']
base=base.sort_values(['uid','create_dt'],ascending=False)
base=base.drop_duplicates(['uid'],keep='first')
base.shape

c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(11099, 6)

In [29]:
#特征衍生
#对连续统计型变量进行函数聚合
#方法包括对历史特征值计数，求历史特征值大于0的个数，求和，求均值，求最大小值，求方差，求极差等

gn=pd.DataFrame()
for i in agg_list:
    tp=df.groupby('uid').apply(lambda df:len(df[i])).reset_index()
    tp.columns=['uid', i +'_cnt']
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn, tp, on='uid',how='left')
    #求历史特征值大于0的个数
    tp=df.groupby('uid').apply(lambda df:np.where(df[i]>0,1,0).sum()).reset_index()
    tp.columns=['uid',i + '_num']
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn, tp,on='uid',how='left')
    #求和
    tp=df.groupby('uid').apply(lambda df:np.nansum(df[i])).reset_index()
    tp.columns=['uid',i + '_tot']
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn,tp,on='uid',how='left')
    #求平均值
    tp=df.groupby('uid').apply(lambda df:np.nanmean(df[i])).reset_index()
    tp.columns=['uid',i + '_avg']
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn,tp,on='uid',how='left')
    #求最大值
    tp=df.groupby('uid').apply(lambda df:np.nanmax(df[i])).reset_index()
    tp.columns=['uid',i +'_max']
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn,tp,on='uid',how='left')
    #求最小值
    tp=df.groupby('uid').apply(lambda df:np.nanmin(df[i])).reset_index()
    tp.columns=['uid',i + '_min']
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn,tp,on='uid',how='left')
    #求方差
    tp=df.groupby('uid').apply(lambda df:np.nanvar(df[i])).reset_index()
    tp.columns=['uid',i+'_var']
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn,tp,on='uid',how='left')
    #求极差
    tp=df.groupby('uid').apply(lambda df:np.nanmax(df[i])-np.nanmin(df[i])).reset_index()
    tp.columns=['uid',i+'_ran']
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn,tp,on='uid',how='left')

c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: Mean of empty slice
c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: All-NaN axis encountered
c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: All-NaN axis encountered
c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: Degrees of freedom <= 0 for slice.
c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: All-NaN axis encountered
c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: Mean of empty slice
c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: All-NaN axis encountered
c:\users\super\appdata\loca

In [31]:
#查看衍生结果
gn.columns

Index(['uid', 'oil_amount_cnt', 'oil_amount_num', 'oil_amount_tot',
       'oil_amount_avg', 'oil_amount_max', 'oil_amount_min', 'oil_amount_var',
       'oil_amount_ran', 'discount_amount_cnt', 'discount_amount_num',
       'discount_amount_tot', 'discount_amount_avg', 'discount_amount_max',
       'discount_amount_min', 'discount_amount_var', 'discount_amount_ran',
       'sale_amount_cnt', 'sale_amount_num', 'sale_amount_tot',
       'sale_amount_avg', 'sale_amount_max', 'sale_amount_min',
       'sale_amount_var', 'sale_amount_ran', 'amount_cnt', 'amount_num',
       'amount_tot', 'amount_avg', 'amount_max', 'amount_min', 'amount_var',
       'amount_ran', 'pay_amount_cnt', 'pay_amount_num', 'pay_amount_tot',
       'pay_amount_avg', 'pay_amount_max', 'pay_amount_min', 'pay_amount_var',
       'pay_amount_ran', 'coupon_amount_cnt', 'coupon_amount_num',
       'coupon_amount_tot', 'coupon_amount_avg', 'coupon_amount_max',
       'coupon_amount_min', 'coupon_amount_var', 'coupon_amou

In [32]:
#对dstc_lst变量求distinct个数
gc=pd.DataFrame()
for i in count_list:
    tp=df.groupby('uid').apply(lambda df:len(set(df[i]))).reset_index()
    tp.columns=['uid',i+'_dstc']
    if gc.empty:
        gc=tp
    else:
        gc=pd.merge(gc,tp,on='uid',how='left')

In [33]:
#将变量组合在一起
fn=pd.merge(base,gn,on='uid')
fn=pd.merge(fn,gc,on='uid')
fn.shape

(11099, 67)

In [34]:
#merge过程中可能会出现缺失情况，填充缺失值
fn=fn.fillna(0)
fn.head(100)

,uid,create_dt,oil_actv_dt,class_new,bad_ind,dtn,oil_amount_cnt,oil_amount_num,oil_amount_tot,oil_amount_avg,...,payment_coupon_amount_avg,payment_coupon_amount_max,payment_coupon_amount_min,payment_coupon_amount_var,payment_coupon_amount_ran,channel_code_dstc,oil_code_dstc,scene_dstc,source_app_dstc,call_source_dstc
0,B96436391985035703,2018-10-08,2018-10-08,B,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
1,B96436391984693397,2018-10-11,2018-10-11,E,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
2,B96436391977217468,2018-10-17,2018-10-17,B,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
3,B96436391976480892,2018-09-28,2018-09-28,B,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
4,B96436391972106043,2018-10-19,2018-10-19,A,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,B96117370332355190,2018-10-19,2018-10-19,B,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
96,B96117370330101658,2018-10-12,2018-10-12,B,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
97,B96117370330066347,2018-10-01,2018-10-01,D,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
98,B96117370328724350,2018-09-20,2018-09-20,C,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1


In [35]:
#训练决策树模型
x=fn.drop(['uid','oil_actv_dt','create_dt','bad_ind','class_new'],axis=1)
y=fn.bad_ind.copy()
from sklearn import tree
dtree=tree.DecisionTreeRegressor(max_depth=2,min_samples_leaf=500,min_samples_split=5000)
dtree=dtree.fit(x,y)

In [36]:
# 特征构造

import pandas as pd
import numpy as np
data=pd.read_excel(r'E:\MyJPY\Datas\风控模型\textdata.xlsx')
data.head()

,customer_id,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,...,gt3,gt4,gt5,gt6,gt7,gt8,gt9,gt10,gt11,gt12
0,111,9,11.0,12,13,18,10,12,NaN,NaN,...,10,0,18,10,12,NaN,NaN,NaN,NaN,NaN
1,112,11,-11.0,10,10,13,13,10,NaN,NaN,...,10,10,13,13,10,NaN,NaN,NaN,NaN,NaN
2,113,0,11.0,10,12,6,10,0,25.0,10.0,...,10,12,6,10,0,25.0,10.0,NaN,NaN,NaN
3,114,-7,-1.0,9,8,7,0,-19,10.0,11.0,...,10,10,12,0,-19,10.0,11.0,NaN,NaN,NaN
4,115,11,NaN,6,10,0,17,19,10.0,30.0,...,6,10,0,17,19,10.0,30.0,15.0,NaN,NaN


In [37]:
#可以根据这个时间序列进行基于经验的人工特征衍生，例如计算最近P个月特征大于0的月份数
#最近P个月，ft>0的月份数
#ft特征名字，p特征大于0的月份数
def Num(ft,p):
    df=data.loc[:ft+'1':ft+str(p)]
    auto_value=np.where(df>0,1,0).sum(axis=1)
    return ft+'_num'+str(p),auto_value

In [38]:
#计算最近P个月特征ft等于0的月份数
#最近p个月，ft=0的月份数
def zero_cnt(ft,p):
    df=data.loc[:,ft+'1':ft+str(p)]
    auto_value=np.where(df==0,1,0).sum(axis=1)
    return ft+'_zero_cnt'+str(p),auto_value

In [39]:
#计算最近p个月特征ft等于0的月份数
#最近p个月，ft=0的月份数
def zero_cnt(ft,p):
    df=data.loc[:,ft+'1':ft+str(p)]
    auto_value=np.where(df==0,1,0).sum(axis=1)
    return ft+'_zero_cnt'+str(p),auto_value

In [40]:
#计算近p个月特征ft大于0的月份数是否大于等于1
#最近p个月，ft>0的月份数是否>=1
def Evr(ft,p):
    df=data.loc[:ft+'1':ft+str(p)]
    arr=np.where(df>0,1,0).sum(axis=1)
    auto_value=np.where(arr,1,0)
    return ft+'_evr'+str(p),auto_value

In [41]:
#计算最近p个月特征ft的均值
#最近p个月，ft均值
def Avg(ft,p):
    df=data.loc[:,ft+'1':ft+str(p)]
    auto_value=np.namemean(df,axis=1)
    return ft+'_avg'+str(p),auto_value

In [42]:
#计算最近p个月特征ft的和，最大值，最小值
#最近p个月,ft和
def Tot(ft,p):
    df=data.loc[:,ft+'1':ft+str(p)]
    auto_value=np.nansum(df,axis=1)
    return ft+'_tot'+str(p),auto_value

In [43]:
#最近（2,p+1）个月，ft和
def Tot2T(ft,p):
    df=data.loc[:,ft+'2':ft+str(p+1)]
    auto_value=df.sum()
    return ft+'_tot2t'+str(p),auto_value

In [44]:
#最近p个月，ft最大值
def Max(ft,p):
    df=data.loc[:,ft+'1':ft+str(p)]
    auto_value=np.nanmax(df,axis=1)
    return ft+'_max'+str(p),auto_value

In [45]:
#最近p个月，ft最小值
def Min(ft,p):
    df=data.loc[:,ft+'1':ft+str(p)]
    auto_value=np.nanmin(df,axis=1)
    return ft+'_min'+str(p),auto_value

In [ ]:
#其余衍生方法


